<a href="https://colab.research.google.com/github/meetAmarAtGithub/15_Reva_Speech_Analytics/blob/main/Session_2_Ronin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd "/content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics/"

/content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics


In [4]:
ls

 1_SpeechAnalytics.ipynb            session1_violin-origional.wav
 AudioFiles/                       'Session 2_Ronin.ipynb'
 Session_1_Speech_analytics.ipynb   Sesssion2_SpeechAnalytics.ipynb


In [1]:
#Import libraries
import librosa
import numpy as np

In [5]:
# Specify the path to the audio file
audio_path = "session1_violin-origional.wav"

# Load the audio file
audio, sr = librosa.load(audio_path)

In [6]:
audio

array([ 3.6351426e-05,  2.4562608e-04,  3.2114607e-04, ...,
       -9.6683308e-02, -9.8030478e-02, -1.0250133e-01], dtype=float32)

In [7]:
len(audio)

110250

In [8]:
#Min-max normalization
def min_max_normalization(audio):
    minx = np.min(audio)
    maxx = np.max(audio)
    normalised_data = (audio-minx)/(maxx-minx)
    return normalised_data

In [9]:
#Test min-max normalization
mmnd = min_max_normalization(audio)
print('Original data:', audio[:5])
print('Scaled data:', mmnd[:5])

Original data: [3.6351426e-05 2.4562608e-04 3.2114607e-04 1.9103831e-04 2.6651236e-05]
Scaled data: [0.42713943 0.42734385 0.4274176  0.4272905  0.42712995]


In [10]:
#Z-score transform
def z_score_transform(audio):
    mu = np.mean(audio)
    sd = np.std(audio)
    normalised_data = (audio-mu)/sd
    return normalised_data

In [11]:
#Test z-transform
ztd =  z_score_transform(audio)
print(ztd[:5])

[0.00096504 0.00341933 0.004305   0.00277915 0.00085128]


In [12]:
#Peak amplitude normalization
def peak_amplitude_transform(audio):
    peak = max(audio)
    print(peak)
    normalised_data = audio/peak
    return normalised_data

In [13]:
#Test peak amplitude transform
pat = peak_amplitude_transform(audio)
print(pat[:5])

0.5865756
[6.1972278e-05 4.1874580e-04 5.4749305e-04 3.2568403e-04 4.5435296e-05]


**Resampling**

In [14]:
from scipy.interpolate import interp1d

In [15]:
def nearest_neighbor_resample(audio, target_length):
    original_length = len(audio)
    x = np.arange(original_length)
    f = interp1d(x, audio, kind='nearest')
    resampled_audio = f(np.linspace(0, original_length - 1, target_length))
    return resampled_audio

In [16]:
print('Current sample rate: ', sr)
target_sr = int(0.8* sr)
print('Target sample rate: ', target_sr)
nnrs = nearest_neighbor_resample(audio, target_sr)
print(len(audio), len(nnrs))
print(nnrs[:5])

Current sample rate:  22050
Target sample rate:  17640
110250 17640
[ 3.6351426e-05  3.2762761e-04  1.5134944e-04 -1.5314802e-04
 -9.8524135e-05]


In [ ]:
#Audio analysis - Music genre classification

In [32]:
!pip install pyAudioAnalysis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyAudioAnalysis: filename=pyAudioAnalysis-0.3.14-py3-none-any.whl size=41264374 sha256=2b14f3e1bdeec1b7b65c1a2be8d5ebd57f76964fe84b3655dd2b7f445cfbd3f0
  Stored in directory: /root/.cache/pip/wheels/a7/54/73/fa830689c2440d2c81ff175c60e374930ad1607a8881e0f43f
Successfully built pyAudioAnalysis


In [36]:
!pip install eyed3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 kB 19.3 MB/s eta 0:00:00


In [38]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
#Import the required files
from pyAudioAnalysis import MidTermFeatures as aF

In [40]:
#Import base libraries
import numpy as np
import matplotlib.pyplot as plt

#Import classifiers
#Local models
from sklearn.naive_bayes import GaussianNB

#Linear models
from sklearn.linear_model import LogisticRegression

#Non linear models
from sklearn import tree

#Validation libraries
from sklearn.metrics import confusion_matrix


In [50]:
#Load the data
path = "/content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics/AudioFiles"

In [55]:
dirs = [path+"/classical/", path+"/Metal/"] 

In [56]:
dirs

['/content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics/AudioFiles/classical/',
 '/content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics/AudioFiles/Metal/']

In [57]:
#Define classes
class_names = ['classical', 'metal'] 

In [58]:
print(class_names)

['classical', 'metal']


In [59]:
#initialize minimu window, minimum step, window size and step size
m_win, m_step, s_win, s_step = 1, 1, 0.1, 0.05

In [60]:
# segment-level feature extraction:
features = [] 

In [61]:
#Extract the features
for d in dirs: # get feature matrix for each directory (class)
    print(d)
    f, _, fn = aF.directory_feature_extraction(d, m_win, m_step, 
                                                   s_win, s_step) 
    features.append(f)

/content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics/AudioFiles/classical/
Analyzing file 1 of 10: /content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics/AudioFiles/classical/data_music_classical_classical.00012.au.wav
Analyzing file 2 of 10: /content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics/AudioFiles/classical/data_music_classical_classical.00017.au.wav
Analyzing file 3 of 10: /content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics/AudioFiles/classical/data_music_classical_classical.00023.au.wav
Analyzing file 4 of 10: /content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics/AudioFiles/classical/data_music_classical_classical.00027.au.wav
Analyzing file 5 of 10: /content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics/AudioFiles/classical/data_music_classical_classical.00030.au.wav
Analyzing file 6 of 10: /content/gdrive/My Drive/Colab Notebooks/Reva/15_Speech_Analytics/AudioFiles/classical/data_music_classical_class

In [62]:
features[:2]

[array([[1.80389961e-01, 5.83426779e-02, 3.24215393e+00, ...,
         8.16622670e-03, 2.00000000e+02, 9.37499989e-02],
        [2.36084579e-01, 4.51324260e-02, 3.20337700e+00, ...,
         5.92134326e-03, 1.71428571e+02, 1.87499998e-01],
        [1.39812924e-01, 7.41176724e-02, 3.25051884e+00, ...,
         5.08672374e-03, 4.00000000e+02, 1.36363635e-01],
        ...,
        [1.65239444e-01, 2.48579223e-02, 3.17368998e+00, ...,
         6.63576808e-03, 2.40000000e+02, 1.37254901e-01],
        [2.52816020e-01, 3.15424944e-02, 3.04960657e+00, ...,
         2.04765510e-02, 1.50000000e+02, 2.22222220e-01],
        [2.33021540e-01, 3.46355248e-02, 3.20578801e+00, ...,
         9.23241609e-03, 2.00000000e+02, 1.39534882e-01]]),
 array([[2.32165207e-01, 4.08303634e-02, 3.15282509e+00, ...,
         1.26967985e-02, 1.71428571e+02, 1.79999999e-01],
        [1.61682366e-01, 5.12111765e-02, 3.05479382e+00, ...,
         2.85935370e-02, 1.50000000e+02, 2.22222220e-01],
        [2.63026151e-01, 

In [63]:
print(features[0].shape, features[1].shape)

(10, 138) (10, 138)


In [64]:
dir(features)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [65]:
# select 2 features and create feature matrices for the two classes:
f1 = np.array([features[0][:, fn.index('spectral_centroid_mean')],
               features[0][:, fn.index('energy_entropy_mean')]])

In [66]:
f2 = np.array([features[1][:, fn.index('spectral_centroid_mean')],
               features[1][:, fn.index('energy_entropy_mean')]])

In [67]:
X = np.concatenate((f1.T, f2.T), axis = 0)

In [68]:
X[:2]

array([[0.26691139, 3.24215393],
       [0.2960767 , 3.203377  ]])

In [69]:
#Create labels
y = np.concatenate((np.zeros(f1.shape[1]), np.ones(f2.shape[1]))) 

In [70]:
y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [71]:
#Build naive Bayesian model
gnb = GaussianNB()
gnb.fit(X, y)

GaussianNB()

In [72]:
y_pred = gnb.predict(X)

In [73]:
y_pred

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [77]:
#Confusion matrix for NB
y_pred = gnb.predict(X)
confusion_matrix(y, y_pred)

array([[ 9,  1],
       [ 0, 10]])

In [75]:
#Logistic regression model
logreg = LogisticRegression()
logreg.fit(X, y)
y_pred = logreg.predict(X)
confusion_matrix(y, y_pred)

array([[ 9,  1],
       [ 0, 10]])

In [76]:
#Decision tree classifier
dt = tree.DecisionTreeClassifier()
dt.fit(X, y)
y_pred = dt.predict(X)
confusion_matrix(y, y_pred)

array([[10,  0],
       [ 0, 10]])